<a href="https://colab.research.google.com/github/adrianleung60/thelayoff/blob/main/webscraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import glob
import requests
import pandas as pd
import numpy as np
import re
import string
import csv
import time
import random
from datetime import datetime
from google.colab import drive
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#function to find number after a text, used to parse html
def find_number(text, c):
    return re.findall(r'%s(\d+)' % c, text)

def get_div_content(url):
  site = requests.get(url).text
  soup = BeautifulSoup(site)
  content = str(soup.find_all('div', attrs = {'id':'content'}))
  return BeautifulSoup(content)

#Section to extract all company sites

In [ ]:
# get company indexes
numindex=list(range(0,10))
for x in numindex:
  numindex[x] = str(numindex[x])

alphindex=list(string.ascii_lowercase)
alphindex1 = alphindex[:len(alphindex)//2]
alphindex2 = alphindex[len(alphindex)//2:]

index = numindex + alphindex1 + alphindex2
alphindex1

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm']

In [ ]:
customindex=['a','b','c','d','e','f','g']
customindex2=['f','g','h','i','j','k','l','m']
customindex3=['l','m','n']

In [ ]:
# get list of all active companies
pd.set_option('display.max_rows', None)
urls = pd.DataFrame(data = siteurls[41:363], columns=['URL']) #extract 41-363 for most active companies
print(urls)

In [ ]:
#get list of URL from all companies
url = 'https://www.thelayoff.com'
siteurls=[]

for x in range(len(index)):
  indexurl = url + "/index/" + index[x]
  contentsoup = get_div_content(indexurl)

  #check if there are additional pages
  flag = len(contentsoup.find('ul', attrs={'class':'pagination'}))

  if flag == 1: #loop through additional pages if they exist
    pageindex = []
    for link in contentsoup.find_all('a'):
      pageindex.append(url + link.get('href'))

    #remove duplicate page urls
    pageindex = list(dict.fromkeys(pageindex))

    #loop through companies in pages
    for y in range(len(pageindex)):
      pageurl = pageindex[y]
      pagecontentsoup = get_div_content(pageurl)
      for link in pagecontentsoup.find_all('a'):
        siteurls.append(url + link.get('href'))

  else: #output links for indexes with no additional pages
    for link in contentsoup.find_all('a'):
      siteurls.append(url + link.get('href'))

# df = pd.DataFrame(data=siteurls, columns=['url'])
# df.to_csv('sites.csv')

In [ ]:
df = pd.DataFrame(data=siteurls, columns=['url'])
df.to_csv('sites.csv')

In [ ]:
# merge all csv into one dataframe

import glob
path = '/content'
all_files = glob.glob(path + "/*.csv")
li=[]

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
df = df.drop(['Unnamed: 0'], axis=1)
df.shape

(41739, 1)

In [ ]:
df.sort_values(by=['url'], ignore_index=True)

,url
0,https://www.thelayoff.com/-eleven
1,https://www.thelayoff.com/01-communique-labora...
2,https://www.thelayoff.com/1-800-radiator
3,https://www.thelayoff.com/1-source-consulting
4,https://www.thelayoff.com/1000bulbs
...,...
41734,https://www.thelayoff.com/zymeworks
41735,https://www.thelayoff.com/zymogenetics
41736,https://www.thelayoff.com/zynerba-pharmaceuticals
41737,https://www.thelayoff.com/zynga


In [ ]:
duplicatedRowsDF = df[df.duplicated()]
print(duplicatedRowsDF)

                                                     url
8049    https://www.thelayoff.com/l-1-identity-solutions
8050   https://www.thelayoff.com/l-3-communications-h...
8051        https://www.thelayoff.com/l-and-h-industrial
8052   https://www.thelayoff.com/l-anse-area-schools-...
8053   https://www.thelayoff.com/l-anse-creuse-public...
...                                                  ...
41734    https://www.thelayoff.com/firehouse-productions
41735  https://www.thelayoff.com/firelands-local-scho...
41736  https://www.thelayoff.com/firelands-regional-m...
41737     https://www.thelayoff.com/firestream-worldwide
41738  https://www.thelayoff.com/firm-capital-mortgag...

[4270 rows x 1 columns]


In [ ]:
dfsites = df.drop_duplicates(keep='first', ignore_index=True)
dfsites.shape

(37469, 1)

In [ ]:
dfsites.sort_values(by=['url'], ignore_index=True, inplace=True)
dfsites.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,url
0,https://www.thelayoff.com/-eleven
1,https://www.thelayoff.com/01-communique-labora...
2,https://www.thelayoff.com/1-800-radiator
3,https://www.thelayoff.com/1-source-consulting
4,https://www.thelayoff.com/1000bulbs


In [ ]:
dfsites.to_csv('allsites.csv')

#Loop through all sites - Create subset for sites with no next page

In [ ]:
dfsites = pd.read_csv('drive/MyDrive/allsites.csv')
sitelist = dfsites.url
sitelist

0                        https://www.thelayoff.com/-eleven
1        https://www.thelayoff.com/01-communique-labora...
2                 https://www.thelayoff.com/1-800-radiator
3            https://www.thelayoff.com/1-source-consulting
4                      https://www.thelayoff.com/1000bulbs
                               ...                        
37464                  https://www.thelayoff.com/zymeworks
37465               https://www.thelayoff.com/zymogenetics
37466    https://www.thelayoff.com/zynerba-pharmaceuticals
37467                      https://www.thelayoff.com/zynga
37468                     https://www.thelayoff.com/zywave
Name: url, Length: 37469, dtype: object

In [ ]:
sitelist[20000]

'https://www.thelayoff.com/marshall-medical-center-lewisburg'

In [ ]:
dfsites[dfsites['url'].str.contains('imperial-oil')]

,Unnamed: 0,url
15733,15733,https://www.thelayoff.com/imperial-oil


In [ ]:
user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
]

In [ ]:
company = []
industry = []
OnePage = []

for x in tqdm(range(20001,37468)):

  user_agent = random.choice(user_agent_list)
  headers = {'User-Agent': user_agent}

  companyurl = sitelist[x] #get url
  companysite = requests.get(companyurl, headers=headers).text #get html
  companysoup = BeautifulSoup(companysite)

  # get company and industry
  try: companystr = companysoup.find('aside', attrs={'id':'company'}).h2.text.replace(' company profile',"")
  except Exception: companystr = companyurl
  try:
    industrystr = companysoup.find('dl', attrs={'class':'header-profile'}).dd.text
  except Exception: industrystr = None

  # check for next site
  next = companysoup.find('li', attrs={'class':'next'})
  if next == None:
    OnePageBool = "Yes"
  else:
    OnePageBool = "No"

  company.append(companystr)
  industry.append(industrystr)
  OnePage.append(OnePageBool)

  time.sleep(0.9)

  if x % 2500 == 0:
    data = {'company':company, 'industry':industry, 'OnePageBool':OnePage}
    tempdata = pd.DataFrame(data)
    tempdata.to_csv('drive/MyDrive/companydata' + str(x) + '.csv')

data = {'company':company, 'industry':industry, 'OnePageBool':OnePage}
tempdata = pd.DataFrame(data)
tempdata.to_csv('drive/MyDrive/companydata' + str(x) + '.csv')


In [ ]:
  data = {'company':company, 'industry':industry, 'OnePageBool':OnePage}
  tempdata = pd.DataFrame(data)
  #tempdata.to_csv('companydata' + str(x) + '.csv')
  tempdata.head(10)

,company,industry,OnePageBool
0,Imperial Holdings Inc.,"Financials // Finance, Special Products",Yes
1,Imperial Industries Inc.,Heavy Machinery & Materials,Yes
2,Imperial Metals Corporation,III,Yes
3,Imperial Oil Limited,IMO,No
4,Imperial Sugar Co.,Food & Tobacco,Yes
5,Imperial Tobacco Group PLC,Food & Tobacco,Yes
6,Imperial Unified School District,"Local and State Government, education and hosp...",Yes
7,Imperial Valley ROP JPA,"Local and State Government, education and hosp...",Yes
8,Imperva Inc.,Technology // Software and Intellectual Property,Yes
9,Impinj,Technology//Semiconductors,Yes


In [ ]:
print(companysoup.prettify())

In [ ]:
# get data file names

path = '/content'
all_files = glob.glob(path + "/*.csv")
li=[]

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)
df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
dfnew.head()

,company,industry,OnePageBool,url
0,"7-Eleven, Inc.",Convenience Stores,Yes,https://www.thelayoff.com/-eleven
1,01 Communique Laboratory,Software,Yes,https://www.thelayoff.com/01-communique-labora...
2,1-800-Radiator,"Retail, Retail Sales, Retailing",Yes,https://www.thelayoff.com/1-800-radiator
3,1 Source Consulting,Government and Local Government services,Yes,https://www.thelayoff.com/1-source-consulting
4,1000Bulbs.com,Consumer Products and Services,Yes,https://www.thelayoff.com/1000bulbs


In [ ]:
dfnew.loc[len(dfnew.index)] = ['Zywave', 'Management and Business Services', 'Yes']  

In [ ]:
df3 = pd.DataFrame(sitelist, columns=['url'])
df3.shape

(37469, 1)

In [ ]:
x = 30000
y = 37469
print(dfnew[x:y])
print(df3[x:y])

                              company  ...                                                url
30000                  Signature Bank  ...           https://www.thelayoff.com/signature-bank
30001           Signature Consultants  ...    https://www.thelayoff.com/signature-consultants
30002                 Signature Foods  ...          https://www.thelayoff.com/signature-foods
30003  Signature Genomic Laboratories  ...  https://www.thelayoff.com/signature-genomic-la...
30004   Signature Group Holdings Inc.  ...  https://www.thelayoff.com/signature-group-hold...
...                               ...  ...                                                ...
37464                       Zymeworks  ...                https://www.thelayoff.com/zymeworks
37465               ZymoGenetics Inc.  ...             https://www.thelayoff.com/zymogenetics
37466         Zynerba Pharmaceuticals  ...  https://www.thelayoff.com/zynerba-pharmaceuticals
37467                           Zynga  ...                  

In [ ]:
dfnew['url'] = df3

In [ ]:
dfnew.to_csv('allcompanies.csv')

#Exploration of data from the site

In [ ]:
df = pd.read_csv('drive/MyDrive/allcompanies.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df.head()

,company,industry,OnePageBool,url
0,"7-Eleven, Inc.",Convenience Stores,Yes,https://www.thelayoff.com/-eleven
1,01 Communique Laboratory,Software,Yes,https://www.thelayoff.com/01-communique-labora...
2,1-800-Radiator,"Retail, Retail Sales, Retailing",Yes,https://www.thelayoff.com/1-800-radiator
3,1 Source Consulting,Government and Local Government services,Yes,https://www.thelayoff.com/1-source-consulting
4,1000Bulbs.com,Consumer Products and Services,Yes,https://www.thelayoff.com/1000bulbs


In [ ]:
df.shape[0]

37469

In [ ]:
industries = df['industry'].value_counts()
numindustries = 20
industries[:numindustries]
print('Percent of total data:', "{:.1%}".format(int(industries[:numindustries].sum())/df.shape[0]))
print('Top', numindustries, 'Industries Represented')
print(industries[:numindustries])

Percent of total data: 73.4%
Top 20 Industries Represented
Local and State Government, education and hospitals ||| School Districts    12547
Health-Care Providers // Acute Care Hospitals                                3365
County government employeer.                                                 3075
Higher education, colleges and universities.                                 1670
Health-Care Providers // Critical Access Hospitals                           1336
Local and State Government, excluding education and hospitals                 984
Information Technology Services                                               544
Banks                                                                         507
Management and Business Services                                              496
Contracting, Building and Construction                                        396
Good production, manufacturing                                                331
Advertising, Marketing, Ads, Promotion 

# Create dataframe for companies with more than one page

In [ ]:
dfno = df[df['OnePageBool'] == "No"].reset_index(drop=True)

In [ ]:
#395 companies with more than 1 page
dfno.shape

(395, 4)

In [ ]:
dfno.head()

,company,industry,OnePageBool,url
0,2U Inc.,Technology // Software and Intellectual Property,No,https://www.thelayoff.com/2u
1,3M,General Industrials,No,https://www.thelayoff.com/a-3m
2,Academy Sports & Outdoors,Specialty Retailers & Services,No,https://www.thelayoff.com/academy-sports-and-o...
3,Accenture,Business & Industrial Services,No,https://www.thelayoff.com/accenture
4,ADT Corp.,"Industrials // Services, Business Support",No,https://www.thelayoff.com/adt


In [ ]:
dfno[dfno['industry'].str.contains('Oil')]

,company,industry,OnePageBool,url
16,Anadarko Petroleum Corp.,Oil & Gas Producers,No,https://www.thelayoff.com/anadarko-petroleum
18,Apache Corp.,Oil & Gas Producers,No,https://www.thelayoff.com/apache
30,Baker Hughes,"Oil Equipment, Services & Distribution",No,https://www.thelayoff.com/baker-hughes
47,BP PLC,Oil & Gas Producers,No,https://www.thelayoff.com/bp-plc
55,Cameron International Corp.,"Oil Equipment, Services & Distribution",No,https://www.thelayoff.com/cameron-international
65,Cenovus Energy,Oil and Natural Gas,No,https://www.thelayoff.com/cenovus-energy
68,CGG Veritas,"Oil Equipment, Services & Distribution",No,https://www.thelayoff.com/cgg-veritas
70,Chaparral Energy,Oil & Gas Producers,No,https://www.thelayoff.com/chaparral
74,Chesapeake Energy Corp.,Oil & Gas Producers,No,https://www.thelayoff.com/chesapeake-energy
75,Chevron Corp.,Oil & Gas Producers,No,https://www.thelayoff.com/chevron


In [ ]:
dfno[dfno['company'].str.contains('Trans')]

,company,industry,OnePageBool,url
32222,Swift Transportation Co.,"Industrials // Transportation, Trucking",No,https://www.thelayoff.com/swift-transportation-co
33465,TransCanada Corp.,"Oil Equipment, Services & Distribution",No,https://www.thelayoff.com/transcanada


In [ ]:
dfno.at[21657,'industry'] = 'Health-Care Providers'

In [ ]:
industries = dfno['industry'].value_counts()
numindustries = 20
industries[:numindustries]
print('Percent of total data:', "{:.1%}".format(int(industries[:numindustries].sum())/dfno.shape[0]))
print('Top', numindustries, 'Industries Represented')
print(industries[:numindustries])

Percent of total data: 60.3%
Top 20 Industries Represented
Oil & Gas Producers                             23
Specialty Retailers & Services                  21
Oil Equipment, Services & Distribution          16
Software                                        15
Computers & Office Equipment                    15
Retailers: Broad Line & Apparel                 14
Business & Industrial Services                  14
Banks                                           12
Higher education, colleges and universities.    12
Aerospace & Defense                             12
Internet & Computer Services                    12
Broadcasting & Entertainment                    10
Telecommunications Equipment                    10
Health-Care Providers                            9
Retailers: Food & Drug                           9
Advertising & Publishing                         7
Heavy Machinery & Materials                      7
Food & Tobacco                                   7
Industrial Transportati

In [ ]:
numindarray = []
percentindarray = []
derivative = []

for x in range(0,100):
  numindarray.append(x)
  percentindarray.append(int(industries[:x].sum())/dfno.shape[0])

  if x == 0:
    derivative.append(0)
  else:
    der = (percentindarray[x] - percentindarray[x-1]) / (numindarray[x] - numindarray[x-1])
    derivative.append(der)
    
fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Line(x=numindarray, y=percentindarray),
    row=1, col=1
)

fig.add_trace(
    go.Line(x=numindarray, y=derivative),
    row=1, col=2
)

fig.update_layout(height=600, width=1200, title_text="Side By Side Subplots")
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [ ]:
industries.shape

(107,)

In [ ]:
numindustries = 20

indexarray =[]
for x in range(0, numindustries):
  indexarray.append(x)

dfindustrycount = pd.DataFrame(industries[:numindustries])
dfindustrycount.columns = ['Num Companies']
dfindustrycount['index'] = indexarray
dfindustrycount['Top Industries'] = dfindustrycount.index
dfindustrycount.set_index('index')
px.bar(dfindustrycount, x='Top Industries', y='Num Companies', height=800, width = 1200, 
       title='Top Industries Represented on TheLayoff.com')

In [ ]:
px.pie(dfindustrycount,names='Top Industries', values='Num Companies', width = 1000)

In [ ]:
dfno.to_csv('drive/MyDrive/dfno.csv')

#This section extracts date time and number of views

In [ ]:
dfno = pd.read_csv('drive/MyDrive/dfno.csv')

In [ ]:
dfno.head()

,Unnamed: 0,company,industry,OnePageBool,url
0,0,2U Inc.,Technology // Software and Intellectual Property,No,https://www.thelayoff.com/2u
1,1,3M,General Industrials,No,https://www.thelayoff.com/a-3m
2,2,Academy Sports & Outdoors,Specialty Retailers & Services,No,https://www.thelayoff.com/academy-sports-and-o...
3,3,Accenture,Business & Industrial Services,No,https://www.thelayoff.com/accenture
4,4,ADT Corp.,"Industrials // Services, Business Support",No,https://www.thelayoff.com/adt


In [ ]:
dfno.iloc[30]

Unnamed: 0                                         30
company                                  Baker Hughes
industry       Oil Equipment, Services & Distribution
OnePageBool                                        No
url            https://www.thelayoff.com/baker-hughes
Name: 30, dtype: object

In [ ]:
views =[]
datetimearr=[]
company=[]

In [ ]:
#loop through all top websites, create DF with company, postdate, and postviews

user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
]

for x in tqdm(range(221,dfno.shape[0])):

  user_agent = random.choice(user_agent_list)
  headers = {'User-Agent': user_agent}

  companyurl = dfno.at[x,'url']
  companyname = dfno.at[x,'company']
  y=0
  endofsite=False

  while (endofsite == False):
    if y == 0:
      companysite = requests.get(companyurl, headers=headers)
    else:
      companysite = requests.get(companyurl+'?page='+ str(y+1), headers=headers)
    y=y+1

    pagesoup = BeautifulSoup(companysite.text)

    if pagesoup.title.text == '404 Not Found':
      endofsite=True

    for view in pagesoup.find_all('span', attrs={'class':'views'}):
      views.append(find_number(str(view), '>')[0])

    for time1 in pagesoup.find_all('span', attrs='post-timeago'):
      datetimearr.append(find_number(str(time1), 'data-timestamp="')[0])
      company.append(companyname)
  
    time.sleep(1)

    if x % 20 == 0:
      data = {'company':company, 'date':datetimearr, 'views':views}
      tempdata = pd.DataFrame(data)
      tempdata.to_csv('drive/MyDrive/posttime' + str(x) + '.csv')

dfpoststime = pd.DataFrame( {'company':company, 'date':datetimearr, 'views':views})
dfpoststime.to_csv('drive/MyDrive/dfpoststime.csv')

#Explore data for posts from top companies

In [ ]:
df = pd.read_csv('drive/MyDrive/dfallposts.csv')
df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
df.head()

,company,date,views
0,2U Inc.,1607652170,356
1,2U Inc.,1604436037,345
2,2U Inc.,1603111881,229
3,2U Inc.,1602291578,212
4,2U Inc.,1601560418,230


In [ ]:
#convert epoch date to year/month/day
datetime.fromtimestamp(int(df['date'][0])).strftime("%Y-%m-%d")

'2020-12-11'

In [ ]:
pd.to_datetime(df['date'])

0        1970-01-01 00:00:01.607652170
1        1970-01-01 00:00:01.604436037
2        1970-01-01 00:00:01.603111881
3        1970-01-01 00:00:01.602291578
4        1970-01-01 00:00:01.601560418
                      ...             
233357   1970-01-01 00:00:01.276283686
233358   1970-01-01 00:00:01.276283611
233359   1970-01-01 00:00:01.276283576
233360   1970-01-01 00:00:01.275507149
233361   1970-01-01 00:00:01.258243200
Name: date, Length: 233362, dtype: datetime64[ns]

In [ ]:
print('number of posts on the company')
print(df['company'].value_counts())

number of posts on the company
Verizon Communications Inc.     9785
Corinthian Colleges Inc.        9206
AT&T                            8748
Qualcomm Inc.                   8602
Intel Corp.                     7687
                                ... 
C.H. Robinson Worldwide Inc.       5
Xylem Inc.                         5
Huntsman Corp.                     5
EXL Service Holdings Inc.          4
Nucor Corp.                        3
Name: company, Length: 395, dtype: int64


In [ ]:
numcompanies = 50

indexarray =[]
for x in range(0, numcompanies):
  indexarray.append(x)

dfpostcount = pd.DataFrame(postcount[:numcompanies])
dfpostcount.columns = ['Num Posts']
dfpostcount['index'] = indexarray
dfpostcount['Company'] = dfpostcount.index
dfpostcount.set_index('index')
px.bar(dfpostcount, x='Company', y='Num Posts', height=800, width = 1200, 
       title='Top ' + str(numcompanies) + ' Companies Represented on TheLayoff.com by posts')

In [ ]:
dfviewcount = df.groupby(['company']).sum()
dfviewcount = dfviewcount.sort_values(by='views', ascending=False)
dfviewcount['index'] = np.arange(0,dfviewcount.shape[0])
dfviewcount['Company'] = dfviewcount.index
dfviewcount.set_index('index', inplace=True)
dfviewcount = dfviewcount.drop(['date'], axis=1)
dfviewcount

,views,Company
index,,
0,15087616,Verizon Communications Inc.
1,14008559,AT&T
2,12154792,Intel Corp.
3,9499753,Cisco Systems Inc.
4,9297341,Qualcomm Inc.
...,...,...
390,3322,Huntsman Corp.
391,3271,Noodles & Co.
392,3066,Regal Entertainment Group


In [ ]:
numtopviews=50
px.bar(dfviewcount.head(numtopviews), x='Company', y='views', height=800, width = 1200, 
       title='Top ' + str(numtopviews) + ' Companies Represented on TheLayoff.com by views')